## Pipeline Parameters

In [0]:
# Audit Parameters
dbutils.widgets.text("job_id", "")
dbutils.widgets.text("job_name", "")
dbutils.widgets.text("job_start_date", "")
dbutils.widgets.text("job_start_datetime", "")
dbutils.widgets.text("task_run_id", "")
dbutils.widgets.text("task_name", "")

In [0]:
%run ../utils/loggers

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv

In [0]:
load_dotenv()

catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

source_table_name = ['kdayno_silver_SP500_stock_prices'
                     , 'kdayno_bronze_SP500_stock_financials'
                     , 'kdayno_bronze_sp500_companies']
target_table_name = 'kdayno_silver_SP500_stock_financials'

# Audit Variables
job_id = dbutils.widgets.get('job_id')
job_name = dbutils.widgets.get('job_name')
job_start_date = dbutils.widgets.get('job_start_date')
job_start_datetime = dbutils.widgets.get('job_start_datetime')
task_run_id = dbutils.widgets.get('task_run_id')
task_name = dbutils.widgets.get('task_name')

## Pipeline Logging


In [0]:
audit_logger(job_id, job_name, 'None', job_start_date, job_start_datetime, task_run_id,  task_name, source_table_name, target_table_name)

etl_logger = etl_logger()

## ETL

In [0]:
silver_SP500_stock_financials_df = spark.sql(
    f"""
    WITH SP500_stock_prices AS (
        SELECT 
            ticker_symbol
            , ROUND(AVG((open_price + close_price + highest_price + lowest_price)) / 4, 2) AS latest_average_trading_price
        FROM {catalog_name}.{schema_name}.{source_table_name[0]}
        WHERE trading_date = (SELECT max(trading_date) FROM {catalog_name}.{schema_name}.{source_table_name[0]})
        GROUP BY ticker_symbol
    ),
    
    SP500_stock_financials AS (
        SELECT *
        FROM {catalog_name}.{schema_name}.{source_table_name[1]}
    ),

    SP500_stock_financial_ratios AS (
        SELECT 
            p.ticker_symbol
            , p.latest_average_trading_price
            , c.company_name
            , c.gics_sector
            , c.gics_sub_industry
            , f.start_date
            , f.end_date
            , f.filing_date
            , f.fiscal_period
            , f.fiscal_year
            , f.diluted_earnings_per_share
            , f.net_income_loss
            , f.equity
            , f.long_term_debt
            , f.diluted_average_shares
            , p.latest_average_trading_price / f.diluted_earnings_per_share AS price_to_earnings_ratio
            , f.net_income_loss / f.equity AS return_on_equity_ratio
            , f.long_term_debt / f.equity AS debt_to_equity_ratio
            , p.latest_average_trading_price * f.diluted_average_shares AS market_capitalization

        FROM  SP500_stock_prices p
        LEFT JOIN SP500_stock_financials f ON p.ticker_symbol = f.ticker_symbol
        LEFT JOIN {catalog_name}.{schema_name}.{source_table_name[2]} c ON p.ticker_symbol = c.ticker_symbol
    )

    SELECT *
    FROM SP500_stock_financial_ratios                    
    """)

In [0]:
etl_logger.info(f'Loading: {silver_SP500_stock_financials_df.count()} rows to: {target_table_name}')

(silver_SP500_stock_financials_df.write.format("delta")
                           .mode("overwrite")
                           .saveAsTable(f'{catalog_name}.{schema_name}.{target_table_name}'))